# 1. Definiowanie Zmiennych

Zmienne definiuje się w plikach konfiguracyjnych za pomocą bloku `variable`. Przykłady różnych typów zmiennych są podane poniżej.

#### Plik `variables.tf`
```hcl
variable "region" {
  type        = string
  description = "Region AWS"
  default     = "us-west-2"
}

variable "instance_count" {
  type        = number
  description = "Liczba instancji EC2"
  default     = 3
}

variable "enable_logging" {
  type        = bool
  description = "Czy włączyć logowanie"
  default     = true
}

variable "instance_types" {
  type        = list(string)
  description = "Lista typów instancji EC2"
  default     = ["t2.micro", "t2.small", "t2.medium"]
}

variable "tags" {
  type        = map(string)
  description = "Tagi dla zasobów AWS"
  default     = {
    Environment = "development"
    Project     = "TerraformExample"
  }
}

variable "admin_user" {
  type = object({
    username = string
    email    = string
  })
  description = "Informacje o administratorze"
  default = {
    username = "admin"
    email    = "admin@example.com"
  }
}

variable "config_values" {
  type = tuple([string, number, bool])
  description = "Przykładowa krotka konfiguracyjna"
  default = ["config", 10, true]
}
```

### 2. Przypisywanie Wartości Zmiennym

Istnieje kilka sposobów na przypisanie wartości zmiennym w Terraformie:

#### a. Plik `terraform.tfvars` lub `*.auto.tfvars`

Można utworzyć plik `terraform.tfvars` lub pliki z rozszerzeniem `.auto.tfvars`, które automatycznie zostaną załadowane przez Terraform.

```hcl
# terraform.tfvars
region        = "us-east-1"
instance_count = 5
enable_logging = false
instance_types = ["t2.micro", "t2.small"]
tags = {
  Environment = "production"
  Project     = "MyProject"
}
admin_user = {
  username = "new_admin"
  email    = "new_admin@example.com"
}
config_values = ["new_config", 20, false]
```

#### b. Flaga wiersza poleceń

Wartości zmiennych można przekazać podczas wywoływania komend Terraform za pomocą flagi `-var`.

```sh
terraform apply -var="region=us-east-1" -var="instance_count=5"
```

#### c. Zmienne środowiskowe

Zmienne mogą być ustawione jako zmienne środowiskowe, prefiksując nazwę zmiennej przez `TF_VAR_`.

```sh
export TF_VAR_region="us-east-1"
export TF_VAR_instance_count=5
terraform apply
```

#### d. Plik `override.tf` lub opcja `-var-file`

Można stworzyć plik `override.tf` lub użyć opcji `-var-file` do załadowania wartości z pliku.

```sh
terraform apply -var-file="custom.tfvars"
```

### 3. Używanie Zmiennych w Konfiguracji

Zmiennych używa się w konfiguracjach Terraforma poprzez interpolację (użycie wyrażeń w ramach `${}`).

#### Plik `main.tf`
```hcl
provider "aws" {
  region = var.region
}

resource "aws_instance" "example" {
  count         = var.instance_count
  ami           = "ami-12345678"
  instance_type = element(var.instance_types, count.index)

  tags = merge(var.tags, {
    Name = "ExampleInstance-${count.index}"
  })

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_s3_bucket" "logging_bucket" {
  count = var.enable_logging ? 1 : 0

  bucket = "logging-bucket"
  acl    = "private"
}

output "admin_username" {
  value = var.admin_user.username
}

output "config_values" {
  value = var.config_values
}
```

### Podsumowanie

W Terraformie zmienne są definiowane za pomocą bloku `variable` i mogą mieć różne typy, takie jak `string`, `number`, `bool`, `list`, `map`, `object` oraz `tuple`. Wartości zmiennym można przypisać za pomocą plików `tfvars`, flag wiersza poleceń, zmiennych środowiskowych, plików `override.tf` lub opcji `-var-file`. Zmiennych używa się w konfiguracjach poprzez interpolację, co pozwala na dynamiczne definiowanie parametrów zasobów.

Użycie zmiennych w ten sposób umożliwia łatwiejsze zarządzanie i modyfikowanie infrastruktury, zwiększając elastyczność i modularność konfiguracji.

# Przykłady Zmiennych dla Każdego Typu

#### 1. `string`
Zmienne typu `string` przechowują tekst.

```hcl
variable "example_string" {
  type        = string
  description = "Przykład zmiennej typu string"
  default     = "domyślna_wartość"
}
```

#### 2. `number`
Zmienne typu `number` przechowują liczby (całkowite lub zmiennoprzecinkowe).

```hcl
variable "example_number" {
  type        = number
  description = "Przykład zmiennej typu number"
  default     = 42
}
```

#### 3. `bool`
Zmienne typu `bool` przechowują wartości logiczne (prawda/fałsz).

```hcl
variable "example_bool" {
  type        = bool
  description = "Przykład zmiennej typu bool"
  default     = true
}
```

#### 4. `list(type)`
Zmienne typu `list` przechowują listy elementów określonego typu.

```hcl
variable "example_list" {
  type        = list(string)
  description = "Przykład zmiennej typu list"
  default     = ["item1", "item2", "item3"]
}
```

#### 5. `map(type)`
Zmienne typu `map` przechowują mapy (słowniki) klucz-wartość.

```hcl
variable "example_map" {
  type        = map(number)
  description = "Przykład zmiennej typu map"
  default     = {
    key1 = 1
    key2 = 2
    key3 = 3
  }
}
```

#### 6. `object({ key = type, ... })`
Zmienne typu `object` przechowują obiekty z określonymi kluczami i typami wartości.

```hcl
variable "example_object" {
  type = object({
    name = string
    age  = number
  })
  description = "Przykład zmiennej typu object"
  default = {
    name = "John Doe"
    age  = 30
  }
}
```

#### 7. `tuple([type, ...])`
Zmienne typu `tuple` przechowują krotki z określonymi typami elementów.

```hcl
variable "example_tuple" {
  type = tuple([string, number, bool])
  description = "Przykład zmiennej typu tuple"
  default = ["example", 10, true]
}
```

### Kompleksowy Przykład Konfiguracji

Poniżej znajduje się pełna konfiguracja z użyciem różnych typów zmiennych.

#### Plik `variables.tf`
```hcl
variable "region" {
  type        = string
  description = "Region AWS"
  default     = "us-west-2"
}

variable "instance_count" {
  type        = number
  description = "Liczba instancji EC2"
  default     = 3
}

variable "enable_logging" {
  type        = bool
  description = "Czy włączyć logowanie"
  default     = true
}

variable "instance_types" {
  type        = list(string)
  description = "Lista typów instancji EC2"
  default     = ["t2.micro", "t2.small", "t2.medium"]
}

variable "tags" {
  type        = map(string)
  description = "Tagi dla zasobów AWS"
  default     = {
    Environment = "development"
    Project     = "TerraformExample"
  }
}

variable "admin_user" {
  type = object({
    username = string
    email    = string
  })
  description = "Informacje o administratorze"
  default = {
    username = "admin"
    email    = "admin@example.com"
  }
}

variable "config_values" {
  type = tuple([string, number, bool])
  description = "Przykładowa krotka konfiguracyjna"
  default = ["config", 10, true]
}
```

#### Plik `main.tf`
```hcl
provider "aws" {
  region = var.region
}

resource "aws_instance" "example" {
  count         = var.instance_count
  ami           = "ami-12345678"
  instance_type = element(var.instance_types, count.index)

  tags = merge(var.tags, {
    Name = "ExampleInstance-${count.index}"
  })

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_s3_bucket" "logging_bucket" {
  count = var.enable_logging ? 1 : 0

  bucket = "logging-bucket"
  acl    = "private"
}

output "admin_username" {
  value = var.admin_user.username
}

output "config_values" {
  value = var.config_values
}
```

### Wyjaśnienie

- **Zmienne**: Zmienne są zdefiniowane dla różnych typów danych, co pozwala na elastyczne konfigurowanie infrastruktury.
- **Wartości domyślne**: Każda zmienna ma wartość domyślną, która będzie używana, jeśli nie zostanie podana inna wartość.
- **Interpolacja zmiennych**: Wartości zmiennych są używane w konfiguracji zasobów (np. `aws_instance`, `aws_s3_bucket`), co pozwala na dynamiczne definiowanie parametrów zasobów.

Używając zmiennych w ten sposób, możesz łatwo zarządzać i modyfikować swoją infrastrukturę bez konieczności zmieniania logiki w plikach konfiguracyjnych.

# Jak działa `count` i `count.index`

W Terraformie `count.index` jest specjalnym wewnętrznym mechanizmem używanym w kontekście zasobów, gdy korzystamy z argumentu `count` do tworzenia wielu instancji tego samego zasobu. `count.index` reprezentuje bieżący indeks w pętli tworzenia zasobów, umożliwiając dostęp do unikalnych wartości dla każdej iteracji.


Argument `count` pozwala na dynamiczne określenie liczby instancji zasobu, które mają zostać utworzone. Jeśli `count` jest większe niż 1, Terraform utworzy wiele kopii zasobu, a `count.index` będzie miało wartość od 0 do `count-1`, odpowiadającą bieżącej iteracji.

#### Przykład: Tworzenie wielu instancji EC2

Załóżmy, że chcemy utworzyć kilka instancji EC2 w AWS.

#### Plik `variables.tf`
```hcl
variable "instance_count" {
  type        = number
  description = "Liczba instancji EC2"
  default     = 3
}

variable "instance_types" {
  type        = list(string)
  description = "Lista typów instancji EC2"
  default     = ["t2.micro", "t2.small", "t2.medium"]
}
```

#### Plik `main.tf`
```hcl
provider "aws" {
  region = "us-west-2"
}

resource "aws_instance" "example" {
  count         = var.instance_count
  ami           = "ami-12345678"
  instance_type = var.instance_types[count.index]

  tags = {
    Name = "ExampleInstance-${count.index}"
  }
}
```

### Wyjaśnienie

- **count**: W tym przykładzie `count` jest ustawione na wartość `var.instance_count`, co oznacza, że Terraform utworzy tyle instancji EC2, ile wynosi wartość zmiennej `instance_count`.
- **count.index**: `count.index` jest używane do uzyskania bieżącego indeksu iteracji. Wartości `count.index` będą 0, 1, 2, ..., `count-1`. 
  - `instance_type = var.instance_types[count.index]` używa `count.index` do wyboru odpowiedniego typu instancji z listy `instance_types` dla każdej instancji EC2.
  - `Name = "ExampleInstance-${count.index}"` dodaje unikalny tag `Name` do każdej instancji, gdzie `${count.index}` jest zamieniane na bieżący indeks (np. `ExampleInstance-0`, `ExampleInstance-1` itd.).

### Przykład: Tworzenie wielu zasobów z różnymi konfiguracjami

Jeśli chcemy stworzyć różne instancje z różnymi konfiguracjami, `count.index` pozwala nam na dynamiczne przypisanie wartości.

#### Plik `main.tf`
```hcl
provider "aws" {
  region = "us-west-2"
}

resource "aws_instance" "example" {
  count         = var.instance_count
  ami           = "ami-12345678"
  instance_type = var.instance_types[count.index % length(var.instance_types)]

  tags = {
    Name = "ExampleInstance-${count.index}"
    Type = element(var.instance_types, count.index % length(var.instance_types))
  }
}
```

### Wyjaśnienie
- **count.index % length(var.instance_types)**: Używa modulo, aby cyklicznie przechodzić przez listę typów instancji, nawet jeśli `count` jest większe niż liczba elementów w liście `instance_types`.
- **element(var.instance_types, count.index % length(var.instance_types))**: Wybiera element z listy `instance_types` na podstawie bieżącego indeksu cyklicznie.

### Podsumowanie

`count.index` jest niezwykle przydatnym mechanizmem w Terraformie, umożliwiającym dynamiczne i programowalne tworzenie wielu instancji zasobów z różnymi konfiguracjami. Używanie `count` wraz z `count.index` pozwala na elastyczne zarządzanie infrastrukturą, co jest szczególnie ważne w skomplikowanych środowiskach, gdzie potrzeba automatyzacji jest kluczowa.


### Walidacja

```hcl
variable "variable_name" {
  type = string

  validation {
    condition     = condition_expression
    error_message = "Opis błędu, który zostanie wyświetlony, gdy warunek nie zostanie spełniony"
  }
}
```

### Elementy Atrybutu `validation`

- **condition**: Wyrażenie warunkowe (boolowskie), które musi być spełnione, aby wartość zmiennej była uznana za poprawną. Jeśli warunek zwraca `false`, zmienna nie przejdzie walidacji.
- **error_message**: Komunikat błędu, który zostanie wyświetlony użytkownikowi, jeśli wartość zmiennej nie spełni warunku określonego w `condition`.

### Przykłady

#### Przykład 1: Walidacja dla zmiennej typu `string`

Załóżmy, że chcemy, aby zmienna miała co najmniej 3 znaki.

```hcl
variable "username" {
  type = string

  validation {
    condition     = length(var.username) >= 3
    error_message = "Username must be at least 3 characters long."
  }
}
```

#### Przykład 2: Walidacja dla zmiennej typu `number`

Chcemy, aby zmienna była liczbą większą od 0.

```hcl
variable "instance_count" {
  type = number

  validation {
    condition     = var.instance_count > 0
    error_message = "Instance count must be greater than 0."
  }
}
```

#### Przykład 3: Walidacja dla zmiennej typu `list`

Załóżmy, że chcemy, aby lista zawierała co najmniej 1 element.

```hcl
variable "subnet_ids" {
  type = list(string)

  validation {
    condition     = length(var.subnet_ids) > 0
    error_message = "At least one subnet ID must be provided."
  }
}
```

#### Przykład 4: Walidacja dla zmiennej typu `map`

Chcemy, aby mapa zawierała klucz "environment".

```hcl
variable "tags" {
  type = map(string)

  validation {
    condition     = contains(keys(var.tags), "environment")
    error_message = "The tags map must contain an 'environment' key."
  }
}
```

#### Przykład 5: Złożona walidacja dla zmiennej typu `object`

Załóżmy, że mamy obiekt, który musi spełniać określone warunki, na przykład wiek użytkownika musi być większy od 18.

```hcl
variable "user" {
  type = object({
    name = string
    age  = number
  })

  validation {
    condition     = var.user.age > 18
    error_message = "User must be older than 18."
  }
}
```

### Przykład Kompleksowej Konfiguracji

Poniżej znajduje się pełna konfiguracja z użyciem zmiennych i atrybutów walidacji.

#### Plik `variables.tf`
```hcl
variable "region" {
  type        = string
  description = "Region AWS"
  default     = "us-west-2"

  validation {
    condition     = contains(["us-west-1", "us-west-2", "us-east-1", "us-east-2"], var.region)
    error_message = "Region must be one of us-west-1, us-west-2, us-east-1, or us-east-2."
  }
}

variable "instance_count" {
  type        = number
  description = "Liczba instancji EC2"
  default     = 3

  validation {
    condition     = var.instance_count > 0
    error_message = "Instance count must be greater than 0."
  }
}

variable "admin_user" {
  type = object({
    username = string
    email    = string
  })
  description = "Informacje o administratorze"
  default = {
    username = "admin"
    email    = "admin@example.com"
  }

  validation {
    condition     = can(regex("^.+@.+$", var.admin_user.email))
    error_message = "Email must be a valid email address."
  }
}
```

### Wyjaśnienie

- **region**: Walidacja sprawdza, czy wartość zmiennej `region` jest jedną z dopuszczalnych wartości.
- **instance_count**: Walidacja zapewnia, że liczba instancji EC2 jest większa niż 0.
- **admin_user**: Walidacja sprawdza, czy adres email jest poprawny (czy pasuje do wzorca regularnego).

Atrybut `validation` jest potężnym narzędziem w Terraformie, pozwalającym na definiowanie niestandardowych reguł walidacji dla zmiennych. Dzięki temu możesz zapewnić, że wartości wprowadzane do konfiguracji są poprawne i spełniają określone kryteria, co pomaga unikać błędów i problemów z wdrożeniem infrastruktury.

# Walidacja

Atrybut `validation` w bloku `variable` w Terraformie służy do definiowania niestandardowych reguł walidacji dla wartości zmiennych. Dzięki temu można zapewnić, że podane wartości spełniają określone kryteria przed zastosowaniem ich w konfiguracji. Atrybut `validation` składa się z dwóch elementów: `condition` oraz `error_message`.


```hcl
variable "variable_name" {
  type = string

  validation {
    condition     = condition_expression
    error_message = "Opis błędu, który zostanie wyświetlony, gdy warunek nie zostanie spełniony"
  }
}
```

### Elementy Atrybutu `validation`

- **condition**: Wyrażenie warunkowe (boolowskie), które musi być spełnione, aby wartość zmiennej była uznana za poprawną. Jeśli warunek zwraca `false`, zmienna nie przejdzie walidacji.
- **error_message**: Komunikat błędu, który zostanie wyświetlony użytkownikowi, jeśli wartość zmiennej nie spełni warunku określonego w `condition`.

### Przykłady

#### Przykład 1: Walidacja dla zmiennej typu `string`

Załóżmy, że chcemy, aby zmienna miała co najmniej 3 znaki.

```hcl
variable "username" {
  type = string

  validation {
    condition     = length(var.username) >= 3
    error_message = "Username must be at least 3 characters long."
  }
}
```

#### Przykład 2: Walidacja dla zmiennej typu `number`

Chcemy, aby zmienna była liczbą większą od 0.

```hcl
variable "instance_count" {
  type = number

  validation {
    condition     = var.instance_count > 0
    error_message = "Instance count must be greater than 0."
  }
}
```

#### Przykład 3: Walidacja dla zmiennej typu `list`

Załóżmy, że chcemy, aby lista zawierała co najmniej 1 element.

```hcl
variable "subnet_ids" {
  type = list(string)

  validation {
    condition     = length(var.subnet_ids) > 0
    error_message = "At least one subnet ID must be provided."
  }
}
```

#### Przykład 4: Walidacja dla zmiennej typu `map`

Chcemy, aby mapa zawierała klucz "environment".

```hcl
variable "tags" {
  type = map(string)

  validation {
    condition     = contains(keys(var.tags), "environment")
    error_message = "The tags map must contain an 'environment' key."
  }
}
```

#### Przykład 5: Złożona walidacja dla zmiennej typu `object`

Załóżmy, że mamy obiekt, który musi spełniać określone warunki, na przykład wiek użytkownika musi być większy od 18.

```hcl
variable "user" {
  type = object({
    name = string
    age  = number
  })

  validation {
    condition     = var.user.age > 18
    error_message = "User must be older than 18."
  }
}
```

### Przykład Kompleksowej Konfiguracji

Poniżej znajduje się pełna konfiguracja z użyciem zmiennych i atrybutów walidacji.

#### Plik `variables.tf`
```hcl
variable "region" {
  type        = string
  description = "Region AWS"
  default     = "us-west-2"

  validation {
    condition     = contains(["us-west-1", "us-west-2", "us-east-1", "us-east-2"], var.region)
    error_message = "Region must be one of us-west-1, us-west-2, us-east-1, or us-east-2."
  }
}

variable "instance_count" {
  type        = number
  description = "Liczba instancji EC2"
  default     = 3

  validation {
    condition     = var.instance_count > 0
    error_message = "Instance count must be greater than 0."
  }
}

variable "admin_user" {
  type = object({
    username = string
    email    = string
  })
  description = "Informacje o administratorze"
  default = {
    username = "admin"
    email    = "admin@example.com"
  }

  validation {
    condition     = can(regex("^.+@.+$", var.admin_user.email))
    error_message = "Email must be a valid email address."
  }
}
```

### Wyjaśnienie

- **region**: Walidacja sprawdza, czy wartość zmiennej `region` jest jedną z dopuszczalnych wartości.
- **instance_count**: Walidacja zapewnia, że liczba instancji EC2 jest większa niż 0.
- **admin_user**: Walidacja sprawdza, czy adres email jest poprawny (czy pasuje do wzorca regularnego).

Atrybut `validation` jest potężnym narzędziem w Terraformie, pozwalającym na definiowanie niestandardowych reguł walidacji dla zmiennych. Dzięki temu możesz zapewnić, że wartości wprowadzane do konfiguracji są poprawne i spełniają określone kryteria, co pomaga unikać błędów i problemów z wdrożeniem infrastruktury.